In [125]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL

In [126]:
import pandas as pd

In [127]:
#Projections for current year
proj_df = pd.read_csv(GLOBAL.PROJECTIONS_2021, index_col=0)

#ADP for current year and scoring format
adp_df = pd.read_csv(GLOBAL.ADP_2021_HALF_PPR, index_col=0)

#Either Standard, Half PPR, or PPR
FANTASY_SCORING_FORMAT = 'Half PPR'

#VOR dataframe based on scoring format
vor_df = proj_df[['Player', 'Pos', 'Tm', FANTASY_SCORING_FORMAT]]

vor_df = vor_df.rename(columns={
    FANTASY_SCORING_FORMAT: 'Projected Fantasy Total'
})


In [128]:
#Get dataframes of specific pos
rb_proj_df = proj_df[proj_df['Pos'] == 'RB']
wr_proj_df = proj_df[proj_df['Pos'] == 'WR']
te_proj_df = proj_df[proj_df['Pos'] == 'TE']
qb_proj_df = proj_df[proj_df['Pos'] == 'QB']

In [129]:
adp_df['ADP RANK'] = adp_df['AVG'].rank()

adp_df_cutoff = adp_df[:100]

replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}


In [130]:
for _, row in adp_df_cutoff.iterrows():
    position = row['POS'][:2]
    player = row['Player']

    if position in replacement_players:
        replacement_players[position] = player

In [131]:
replacement_values = {}

for position, player_name in replacement_players.items():
    player = vor_df.loc[vor_df['Player'] == player_name]
    replacement_values[position] = player['Projected Fantasy Total'].tolist()[0]

In [132]:
pd.set_option('chained_assignment', None)
pd.set_option('display.max_rows', None)

vor_df = vor_df.loc[vor_df['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]

vor_df['VOR'] = vor_df.apply(
    lambda row: row['Projected Fantasy Total'] - replacement_values.get(row['Pos']), axis=1
)

In [133]:
vor_df['VOR Rank'] = vor_df['VOR'].rank(ascending=False)
# vor_df.sort_values(by='VOR Rank')

In [134]:
min_vor = vor_df['VOR'].min()
max_vor = vor_df['VOR'].max()
vor_range = max_vor - min_vor

vor_df['VOR'] = vor_df['VOR'].apply(
    lambda x: (x - min_vor) / vor_range
)

In [135]:
vor_df = vor_df.sort_values(by= 'VOR', ascending=False)

In [136]:
vor_df = vor_df.rename({
    'VOR': 'Value',
    'VOR Rank': 'Value Rank',
    'Tm': 'Team'
}, axis=1)

In [137]:
adp_df = adp_df[['Player', 'POS', 'Bye', 'AVG', 'ADP RANK']]

adp_df = adp_df.rename(columns={
    'POS': 'Pos Rank',
    'AVG': 'Average ADP',
    'ADP RANK': 'ADP Rank'
})

In [138]:
adp_df['Pos'] = adp_df.apply(
    lambda row: str(row['Pos Rank'])[:2]
, axis=1)

adp_df.head()

,Player,Pos Rank,Bye,Average ADP,ADP Rank,Pos
Rank,,,,,,
1.0,Christian McCaffrey,RB1,13.0,1.0,1.0,RB
2.0,Dalvin Cook,RB2,7.0,2.0,2.0,RB
3.0,Derrick Henry,RB3,13.0,3.0,3.0,RB
4.0,Alvin Kamara,RB4,6.0,4.0,4.0,RB
5.0,Saquon Barkley,RB5,10.0,5.7,5.0,RB


In [139]:
final_df = vor_df.merge(adp_df, how='left', on=['Player', 'Pos'])

In [140]:
final_df['Diff in ADP and Value'] = final_df['ADP Rank'] - final_df['Value Rank']
final_df = final_df.dropna()

In [141]:
# Number of Teams * Spots on Each Team
draft_pool = final_df.sort_values(by='ADP Rank')[:196]

In [142]:
rb_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'RB']
wr_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'WR']
te_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'TE']
qb_df_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'QB']

#Sleepers and Over Valued by Pos.

RBs

Sleepers

In [143]:
rb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
163,Darrel Williams,RB,KAN,69.8,0.394086,164.0,RB74,12.0,233.5,257.5,93.5
176,Sony Michel,RB,NWE,62.0,0.377704,177.0,RB83,14.0,236.5,261.0,84.0
196,Justin Jackson,RB,LAC,49.2,0.350821,197.0,RB71,7.0,239.5,266.0,69.0
170,Joshua Kelley,RB,LAC,66.4,0.386945,170.5,RB81,7.0,221.0,233.5,63.0
188,Malcolm Brown,RB,MIA,53.7,0.360272,189.0,RB66,14.0,230.0,249.5,60.5
113,Tarik Cohen,RB,CHI,99.5,0.456462,114.0,RB52,10.0,170.3,173.0,59.0
95,Phillip Lindsay,RB,HOU,114.4,0.487756,96.0,RB49,10.0,156.7,153.0,57.0
180,Boston Scott,RB,PHI,59.9,0.373294,181.0,RB76,14.0,214.0,225.0,44.0
140,James White,RB,NWE,82.1,0.419919,141.0,RB54,14.0,179.0,185.0,44.0
153,Salvon Ahmed,RB,MIA,76.6,0.408367,154.0,RB61,14.0,186.5,196.0,42.0


Over Valued

In [144]:
rb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
128,Darrell Henderson,RB,LAR,89.7,0.435880,129.0,RB31,11.0,71.7,71.0,-58.0
138,Gus Edwards,RB,BAL,82.4,0.420549,139.0,RB42,8.0,111.3,108.5,-30.5
177,Anthony McFarland Jr.,RB,PIT,61.7,0.377074,178.0,RB62,7.0,164.0,163.5,-14.5
89,Leonard Fournette,RB,TB,119.9,0.499307,90.0,RB32,9.0,78.3,76.0,-14.0
28,Antonio Gibson,RB,WAS,186.9,0.640022,29.0,RB11,9.0,16.3,15.0,-14.0
26,Austin Ekeler,RB,LAC,191.7,0.650103,27.0,RB10,7.0,13.7,14.0,-13.0
37,Clyde Edwards-Helaire,RB,KAN,173.8,0.612509,38.0,RB15,12.0,25.0,25.0,-13.0
40,Miles Sanders,RB,PHI,167.0,0.598227,41.0,RB17,14.0,31.3,30.0,-11.0
126,Nyheim Hines,RB,IND,90.0,0.436510,127.0,RB44,14.0,121.7,117.0,-10.0
88,David Johnson,RB,HOU,121.6,0.502877,89.0,RB34,10.0,85.0,80.0,-9.0


WRs

Sleepers

In [145]:
wr_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
135,Nelson Agholor,WR,NWE,122.7,0.425799,136.0,WR72,14.0,209.5,219.5,83.5
199,Emmanuel Sanders,WR,BUF,85.7,0.348091,200.0,WR81,7.0,240.5,269.0,69.0
189,Parris Campbell,WR,IND,90.7,0.358592,190.0,WR76,14.0,231.5,253.0,63.0
184,A.J. Green,WR,ARI,95.2,0.368043,185.0,WR66,12.0,224.7,242.0,57.0
164,Darius Slayton,WR,NYG,106.4,0.391566,165.0,WR77,10.0,208.0,217.0,52.0
217,DeSean Jackson,WR,LAR,74.7,0.324988,218.5,WR83,11.0,240.0,267.5,49.0
225,Bryan Edwards,WR,LVR,68.8,0.312597,226.0,WR89,8.0,235.0,260.0,34.0
98,Corey Davis,WR,NYJ,151.2,0.485655,99.0,WR48,6.0,130.3,128.0,29.0
173,Jalen Reagor,WR,PHI,102.8,0.384005,174.0,WR68,14.0,191.5,201.0,27.0
59,Courtland Sutton,WR,DEN,184.3,0.555173,60.0,WR32,11.0,88.0,84.0,24.0


Over Valued

In [146]:
wr_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
155,DeVonta Smith,WR,PHI,112.6,0.404587,156.0,WR33,14.0,88.7,85.0,-71.0
213,Christian Kirk,WR,ARI,77.2,0.330239,214.0,WR64,12.0,149.0,148.0,-66.0
223,Kadarius Toney,WR,NYG,69.3,0.313647,224.0,WR70,10.0,173.0,176.0,-48.0
152,Antonio Brown,WR,TB,114.6,0.408787,153.0,WR41,9.0,112.0,110.0,-43.0
187,Sterling Shepard,WR,NYG,93.8,0.365103,188.0,WR63,10.0,148.0,147.0,-41.0
107,Ja'Marr Chase,WR,CIN,142.2,0.466753,108.0,WR27,10.0,72.7,72.0,-36.0
108,Chase Claypool,WR,PIT,140.6,0.463393,109.0,WR28,7.0,74.7,73.0,-36.0
87,Cooper Kupp,WR,LAR,159.4,0.502877,88.0,WR20,11.0,54.3,53.0,-35.0
85,Tyler Lockett,WR,SEA,161.4,0.507078,86.0,WR21,9.0,56.3,54.0,-32.0
212,Russell Gage,WR,ATL,77.6,0.331079,213.0,WR61,6.0,178.0,183.0,-30.0


TEs

Sleepers

In [147]:
te_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
130,Adam Trautman,TE,NOR,81.8,0.433360,131.0,TE27,6.0,232.0,254.5,123.5
148,Eric Ebron,TE,PIT,72.0,0.412778,149.0,TE25,7.0,223.5,238.5,89.5
124,Austin Hooper,TE,CLE,84.0,0.437980,125.0,TE21,13.0,202.0,210.0,85.0
143,Blake Jarwin,TE,DAL,75.0,0.419078,144.0,TE24,7.0,214.7,227.0,83.0
118,Anthony Firkser,TE,TEN,88.0,0.446381,119.0,TE22,13.0,193.0,202.0,83.0
112,Jared Cook,TE,LAC,93.7,0.458353,113.0,TE19,7.0,180.0,188.5,75.5
75,Irv Smith Jr.,TE,MIN,122.2,0.518209,76.0,TE16,7.0,142.3,143.0,67.0
171,Pat Freiermuth,TE,PIT,59.6,0.386735,172.0,TE26,7.0,223.5,238.5,66.5
154,Hayden Hurst,TE,ATL,68.7,0.405847,155.0,TE30,6.0,199.0,207.0,52.0
157,Jimmy Graham,TE,CHI,66.8,0.401857,158.0,TE31,10.0,201.0,208.5,50.5


Over Valued

In [148]:
te_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
82,Kyle Pitts,TE,ATL,120.1,0.513798,83.0,TE4,6.0,51.0,50.0,-33.0
30,Darren Waller,TE,LVR,173.8,0.626580,31.0,TE3,8.0,26.0,27.0,-4.0
105,Rob Gronkowski,TE,TB,99.3,0.470114,106.0,TE11,9.0,108.0,105.0,-1.0
9,Travis Kelce,TE,KAN,242.0,0.769816,10.0,TE1,12.0,10.7,10.0,0.0
73,Noah Fant,TE,DEN,125.5,0.525140,74.0,TE7,11.0,79.0,77.0,3.0
43,Mark Andrews,TE,BAL,158.6,0.594657,44.0,TE5,8.0,51.7,52.0,8.0
15,George Kittle,TE,SFO,209.1,0.700718,16.0,TE2,6.0,25.7,26.0,10.0
79,Logan Thomas,TE,WAS,120.2,0.514008,80.5,TE9,9.0,97.0,92.5,12.0
106,Hunter Henry,TE,NWE,98.0,0.467384,107.0,TE13,14.0,122.3,119.0,12.0
149,Zach Ertz,TE,PHI,71.3,0.411308,150.0,TE17,14.0,164.0,163.5,13.5


QBs

Sleepers

In [149]:
qb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
119,Derek Carr,QB,LVR,243.34,0.443231,120.0,QB28,8.0,188.5,198.5,78.5
111,Tua Tagovailoa,QB,MIA,250.78,0.458857,112.0,QB23,14.0,177.7,181.0,69.0
178,Daniel Jones,QB,NYG,211.08,0.375478,179.0,QB30,10.0,222.3,236.0,57.0
174,Cam Newton,QB,NWE,214.32,0.382283,175.0,QB29,14.0,215.3,228.0,53.0
76,Matt Ryan,QB,ATL,277.96,0.515941,77.0,QB14,6.0,125.0,124.0,47.0
99,Baker Mayfield,QB,CLE,262.62,0.483723,100.0,QB16,13.0,140.7,140.0,40.0
110,Kirk Cousins,QB,MIN,252.04,0.461503,111.0,QB17,7.0,147.0,146.0,35.0
65,Joe Burrow,QB,CIN,293.02,0.547570,66.0,QB13,10.0,103.0,101.0,35.0
45,Jalen Hurts,QB,PHI,312.92,0.589364,46.0,QB10,14.0,83.7,79.0,33.0
18,Lamar Jackson,QB,BAL,360.44,0.689167,19.0,QB4,8.0,47.7,46.5,27.5


Over Valued

In [150]:
qb_df_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True).head(10)

,Player,Pos,Team,Projected Fantasy Total,Value,Value Rank,Pos Rank,Bye,Average ADP,ADP Rank,Diff in ADP and Value
232,Justin Fields,QB,CHI,134.88,0.215441,233.0,QB20,10.0,165.7,165.5,-67.5
238,Trey Lance,QB,SFO,92.56,0.126559,239.0,QB22,6.0,169.7,172.0,-67.0
234,Deshaun Watson,QB,HOU,125.04,0.194775,235.0,QB19,10.0,168.7,170.0,-65.0
237,Taysom Hill,QB,NOR,103.60,0.149746,238.0,QB24,6.0,178.0,183.0,-55.0
202,Ryan Fitzpatrick,QB,WAS,196.60,0.345067,203.0,QB25,9.0,162.0,160.0,-43.0
230,Zach Wilson,QB,NYJ,164.92,0.278532,231.0,QB26,6.0,182.0,193.0,-38.0
186,Ben Roethlisberger,QB,PIT,206.70,0.366279,187.0,QB21,7.0,168.3,169.0,-18.0
147,Trevor Lawrence,QB,JAC,229.58,0.414332,148.0,QB15,7.0,131.7,131.0,-17.0
228,Jared Goff,QB,DET,172.54,0.294535,229.0,QB31,9.0,213.0,222.5,-6.5
84,Matthew Stafford,QB,LAR,274.94,0.509598,85.0,QB11,11.0,93.3,89.0,4.0


In [151]:
final_df = final_df[['Player', 'Pos Rank', 'Team', 'Bye', 'Value', 'Value Rank', 'Average ADP', 'ADP Rank', 'Diff in ADP and Value', 'Projected Fantasy Total']]

In [154]:
final_df.to_csv(r'../FinalData/ValueOverReplacement_{}.csv'.format(FANTASY_SCORING_FORMAT), encoding='utf-8', index=False);